In [1]:
# importing all neccessary modules
import numpy as np
import pandas as pd
import glob
from typing import List, Set
import re
import nltk
from nltk.stem import WordNetLemmatizer
import os
import io
import random


In [5]:
from google.colab import files
uploaded = files.upload()

Saving Preprocessed_raw_baye.rar to Preprocessed_raw_baye.rar


In [6]:
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!unrar x /content/Preprocessed_raw_baye.rar

Streaming output truncated to the last 5000 lines.
Extracting  4787.2002-01-02.kitchen.ham.txt                               86%  OK 
Extracting  4787.2005-04-02.GP.spam.txt                                   86%  OK 
Extracting  4787.2005-05-12.BG.spam.txt                                   86%  OK 
Extracting  4787.2005-06-29.GP.spam.txt                                   86%  OK 
Extracting  4787.2005-07-19.SA_and_HP.spam.txt                            86%  OK 
Extracting  4788.2001-03-19.kaminski.ham.txt                              86%  OK 
Extracting  4788.2001-08-03.farmer.ham.txt                                86%  OK 
Extracting  4788.2002-01-15.williams.ham.txt                              86%  OK 
Extracting  4788.2005-05-12.BG.spam.txt                                   86%  OK 
Extracting  4788.2005-06-11.BG.spam.txt                                   86%  OK 
Extracting  4788.2005-07-19.S

In [2]:
emails = [os.path.join(pth, f) for pth, dirs, files in os.walk('/content') for f in files]

In [3]:
len(emails)

33722

In [5]:
import random
emails = random.sample(emails, 8000)  # Selecting a sample of 15000 from the population data to save RAM space during computation

In [6]:
len(emails)

8000

In [7]:
emails = [email for email in emails if 'ham' or 'spam' in email]

In [8]:
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
# If message is spam return 1 else return 0
target = [1 if 'spam' in email else 0 for email in emails]

In [10]:
len(target)

8000

In [11]:
def format_emails_to_messages(email)->List[str]:
    '''Extracts messages from a file and output them as a list of strings'''
    message = []
    lemma = WordNetLemmatizer()
    with open(email, encoding='utf-8', errors='ignore') as eml:
        for line in eml:
            if not line.startswith('Subject:'):    # Dropping the Subject
                new_line = re.findall(r'\b[^\d\W]+\b', line)  # Dropping words that include numbers and non alphanumeric character
                new_line = [lemma.lemmatize(word) for word in new_line if len(word) >2] # Lemmatizing and dropping words with length less than two
                message += new_line
    return list(set(message))

In [12]:
messages = [format_emails_to_messages(email) for email in emails]  # Extracting the messages from the emails

In [13]:
Total_words = list(set([word for words in messages for word in words])) # Total words without duplicates

In [14]:
def word_count_encoding(document):
    return [document.count(word) if word in document else 0 for word in Total_words]


In [15]:
word_count_vector = [word_count_encoding(document) for document in messages]

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [17]:
X_train, X_test, y_train, y_test = train_test_split(word_count_vector, target, test_size=0.3)

In [18]:
naive_multinomial = MultinomialNB()

In [19]:
naive_multinomial.fit(X_train, y_train)

MultinomialNB()

In [20]:
model_prediction = naive_multinomial.predict(X_test)

In [22]:
accuracy_score(y_test, model_prediction)

0.975

In [24]:
print(confusion_matrix(y_test, model_prediction))

[[1194    9]
 [  51 1146]]
